# Ensemble Learning

## Initial Imports

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [16]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [17]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)
df = df.drop(columns=["home_ownership", "verification_status", "issue_d","hardship_flag", "debt_settlement_flag", "pymnt_plan", "initial_list_status", "next_pymnt_d" , "application_type"])

# Preview the data
df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0.0,3.0,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,0.0,6.0,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,0.0,4.0,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0.0,1.0,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,0.0,1.0,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0


In [25]:
#verify that theres no objects where we dont want them because that gave an error down the line 
df.dtypes

loan_amnt                     float64
int_rate                      float64
installment                   float64
annual_inc                    float64
loan_status                    object
                               ...   
tax_liens                     float64
tot_hi_cred_lim               float64
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
Length: 77, dtype: object

## Split the Data into Training and Testing

In [26]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]

In [27]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [28]:
# Check the balance of our target values
# YOUR CODE HERE
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [29]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape

(51612, 76)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [30]:
# Create the StandardScaler instance
# YOUR CODE HERE
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [31]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [32]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [33]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(random_state=1)

In [34]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = brf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.7113181929890295

In [35]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[   51,    36],
       [ 2800, 14318]], dtype=int64)

In [36]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.59      0.84      0.03      0.70      0.48        87
   low_risk       1.00      0.84      0.59      0.91      0.70      0.50     17118

avg / total       0.99      0.84      0.59      0.91      0.70      0.50     17205



In [37]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = brf.feature_importances_
importances_sorted = sorted(zip(brf.feature_importances_, X.columns), reverse=False)
importances_sorted

[(0.0, 'acc_now_delinq'),
 (0.0, 'collection_recovery_fee'),
 (0.0, 'delinq_amnt'),
 (0.0, 'num_tl_120dpd_2m'),
 (0.0, 'num_tl_30dpd'),
 (0.0, 'policy_code'),
 (0.0, 'recoveries'),
 (0.0, 'tax_liens'),
 (0.000320531105761214, 'chargeoff_within_12_mths'),
 (0.00039693516825594895, 'collections_12_mths_ex_med'),
 (0.0013961103825231335, 'pub_rec_bankruptcies'),
 (0.0014123876736330898, 'num_tl_90g_dpd_24m'),
 (0.0022989374101829575, 'pub_rec'),
 (0.002699399676908053, 'delinq_2yrs'),
 (0.003352579531899221, 'tot_coll_amt'),
 (0.004613727321284211, 'open_il_12m'),
 (0.00466809485182879, 'num_accts_ever_120_pd'),
 (0.0054525584297478915, 'open_rv_12m'),
 (0.0056259039591155776, 'open_acc_6m'),
 (0.0059989305639835365, 'num_tl_op_past_12m'),
 (0.006118492393871269, 'inq_last_6mths'),
 (0.006136781461702092, 'open_rv_24m'),
 (0.0065467063539377335, 'total_rec_late_fee'),
 (0.006977344815476469, 'mort_acc'),
 (0.007247618531462127, 'inq_fi'),
 (0.0075510452011459984, 'percent_bc_gt_75'),
 (0.

### Easy Ensemble Classifier

In [38]:
# Train the Classifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(random_state=3)
eec.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(random_state=3)

In [39]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = eec.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.7392661888473919

In [40]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[   60,    27],
       [ 3614, 13504]], dtype=int64)

In [41]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.69      0.79      0.03      0.74      0.54        87
   low_risk       1.00      0.79      0.69      0.88      0.74      0.55     17118

avg / total       0.99      0.79      0.69      0.88      0.74      0.55     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    the balanced random forest classfier

2. Which model had the best recall score?

    the balanced random forest classfier

3. Which model had the best geometric mean score?

    the balanced random forest classfier

4. What are the top three features?

    not sure